In [5]:
import torch
import random
import numpy as np
import datetime as dt
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torchvision import transforms
from torch.optim.lr_scheduler import MultiStepLR
from utils.util import icarl_cifar100_augment_data, get_dataset_per_pixel_mean

from avalanche.models import IcarlNet, make_icarl_net, initialize_icarl_net
from avalanche.training.supervised import ICaRL
from avalanche.logging import InteractiveLogger, TensorboardLogger
from avalanche.benchmarks.datasets import CIFAR10
from avalanche.benchmarks.generators import nc_benchmark
from avalanche.benchmarks.utils import AvalancheDataset, make_classification_dataset
from avalanche.training.plugins import EvaluationPlugin
from avalanche.training.plugins.lr_scheduling import LRSchedulerPlugin
from avalanche.evaluation.metrics import ExperienceAccuracy, ExperienceLoss, ExperienceForgetting, ExperienceTime, EpochAccuracy, StreamAccuracy

In [6]:
seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
cudnn.enabled = False # cudnn library를 사용하지 않게 만듬 / cudnn (cuda 라이브러리)의 기본 설정이 시드를 고정하지 않기 때문에 고정된 결과를 얻을 수 있습니다. 그런데 cudnn을 사용하지 않으면 속도가 많이 느려집니다.
cudnn.deterministic = True  # 연산 처리 속도 감소 -> 모델과 코드를 배포해야 하는 연구 후반 단계에 사용

In [7]:
train_set = CIFAR10('data/CIFAR100', train=True, download=True)
test_set = CIFAR10('data/CIFAR100', train=False, download=True)

per_pixel_mean = get_dataset_per_pixel_mean(CIFAR10('data/CIFAR100', train=True, download=True, transform=transforms.Compose([transforms.ToTensor()])))

transforms_group = dict(
        eval=(
            transforms.Compose(
                [
                    transforms.ToTensor(),
                    lambda img_pattern: img_pattern - per_pixel_mean,
                ]
            ),
            None,
        ),
        train=(
            transforms.Compose(
                [
                    transforms.ToTensor(),
                    lambda img_pattern: img_pattern - per_pixel_mean,
                    icarl_cifar100_augment_data,
                ]
            ),
            None,
        ),
    )

train_set = CIFAR10('data/CIFAR100', train=True,download=True,)
test_set = CIFAR10('data/CIFAR100', train=False,download=True,)
train_set = make_classification_dataset(train_set, transform_groups=transforms_group, initial_transform_group="train",)
test_set = make_classification_dataset(test_set, transform_groups=transforms_group, initial_transform_group="eval",)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [9]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

num_class = 10
incremental = 1
lr_milestones = [20,30,40,50]
lr_factor = 5.0

scenario = nc_benchmark(train_dataset=train_set,
                        test_dataset=test_set,
                        n_experiences=incremental,
                        task_labels=False,
                        seed=seed,
                        shuffle=False,
                        )

model: IcarlNet = make_icarl_net(num_classes=num_class)
model.apply(initialize_icarl_net)

optimizer = optim.SGD(model.parameters(), lr=2.0, momentum=0.9, weight_decay=1e-5)
sched = LRSchedulerPlugin(MultiStepLR(optimizer, lr_milestones, gamma=1.0 / lr_factor))

In [10]:
date = dt.datetime.now()
date = date.strftime("%Y_%m_%d_%H_%M_%S")

interactive_logger = InteractiveLogger()
tensor_logger = TensorboardLogger("logs_iCaRL_cifar10_" + date)
eval_plugin = EvaluationPlugin(
    EpochAccuracy(),
    ExperienceAccuracy(),
    StreamAccuracy(),
    loggers=[interactive_logger, tensor_logger])

In [11]:
memory_size = 2000
train_batch = 512
eval_batch = 256
epoch = 60

buffer_transform = transforms.Compose([icarl_cifar100_augment_data])

strategies = ICaRL(model.feature_extractor, model.classifier, optimizer, memory_size, buffer_transform=buffer_transform, fixed_memory=True, train_mb_size=train_batch, train_epochs=epoch, eval_mb_size=eval_batch, device=device, plugins=[sched], evaluator=eval_plugin)  # criterion = ICaRLLossPlugin()

In [12]:
for i, exp in enumerate(scenario.train_stream):
    eval_exps = [e for e in scenario.test_stream][: i + 1]
    strategies.train(exp)
    strategies.eval(eval_exps)

-- >> Start of training phase << --
  3%|▎         | 3/98 [00:08<04:12,  2.66s/it]

KeyboardInterrupt: 